In [12]:
import os
import sys
sys.path.append('../src/')
from util import *
from feature_makers import *

In [13]:
prefix = '../data/breast_cancer_paclitaxel_plus_radiation/'
pathto_xpr = 'GSE22513_expression_by_symbol.txt'
pathto_resp = 'responders.txt'
pathto_nonresp = 'nonresponders.txt'
xpressions_pr, responses_pr, target_ids_pr, gens_pr = read_GEO_dataset(pathto_xpr, 
                                                                       pathto_resp,
                                                                       pathto_nonresp,
                                                                       prefix)
mask = range(0,responses_pr.shape[0],2)
xpressions_pr, responses_pr, target_ids_pr = \
xpressions_pr[mask, :], responses_pr[mask], target_ids_pr[mask]

In [14]:
path_to_dataset = "../data/breast_cancer_TFAC/GSE20194_breastCancer_TFAC.txt"
dataset = pd.read_csv(path_to_dataset, sep='\t')
xpressions, responses, target_ids, gens = preprocess_dataset(dataset)

In [15]:
target_pr_descr = """
GSM559042	non-pCR breast biopsy, sample 1 rep 1
GSM559043	non-pCR breast biopsy, sample 1 rep 2
GSM559044	non-pCR breast biopsy, sample 2 rep 1
GSM559045	non-pCR breast biopsy, sample 2 rep 2
GSM559046	non-pCR breast biopsy, sample 3 rep 1
GSM559047	non-pCR breast biopsy, sample 3 rep 2
GSM559048	non-pCR breast biopsy, sample 4 rep 1
GSM559049	non-pCR breast biopsy, sample 4 rep 2
GSM559050	non-pCR breast biopsy, sample 5 rep 1
GSM559051	non-pCR breast biopsy, sample 5 rep 2
GSM559052	non-pCR breast biopsy, sample 6 rep 1
GSM559053	non-pCR breast biopsy, sample 6 rep 2
GSM559054	non-pCR breast biopsy, sample 8 rep 1
GSM559055	non-pCR breast biopsy, sample 8 rep 2
GSM559056	non-pCR breast biopsy, sample 9 rep 1
GSM559057	non-pCR breast biopsy, sample 9 rep 2
GSM559058	non-pCR breast biopsy, sample 10 rep 1
GSM559059	non-pCR breast biopsy, sample 10 rep 2
GSM559060	non-pCR breast biopsy, sample 13 rep 1
GSM559061	non-pCR breast biopsy, sample 13 rep 2
GSM559062	pCR breast biopsy, sample 15 rep 1
GSM559063	pCR breast biopsy, sample 15 rep 2
GSM559064	pCR breast biopsy, sample 16 rep 1
GSM559065	pCR breast biopsy, sample 16 rep 2
GSM559066	pCR breast biopsy, sample 18 rep 1
GSM559067	pCR breast biopsy, sample 18 rep 2
GSM559068	pCR breast biopsy, sample 19 rep 1
GSM559069	pCR breast biopsy, sample 19 rep 2
"""

In [16]:
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import scale

knn = KNeighborsClassifier(n_neighbors=3)
clf = SVC(kernel="linear", probability=True)
xpressions_scaled = scale(xpressions, axis=1)

In [17]:
np.random.seed(358)
samples = gen_samples(xpressions, responses, iters=10, cv=10)

In [28]:
a, b, cv_results_ident, descriptions = do_simmetry_class_cv(xpressions,
                                                            responses,
                                                            samples,
                                                            clf,
                                                            feature_maker=identity_heuristic)
process_results(a, b, cv_results_ident)

iteration: 0
iteration: 1
iteration: 2
iteration: 3
iteration: 4
iteration: 5
iteration: 6
iteration: 7
iteration: 8
iteration: 9


,precision 0,recall 0,precision 1,recall 1,accuracy,auc
схема а,0.797690,0.769333,0.796456,0.788333,0.778833,0.864956
схема б,0.783832,0.767857,0.773316,0.787500,0.777679,0.851706


In [ ]:
a_h4, b_h4, cv_results_h4, descriptions_h4  = do_simmetry_class_cv(xpressions,
                                                                   responses, 
                                                                   samples[:1],
                                                                   clf,
                                                                   feature_maker=heuristic_4_features)
process_results(a_h4, b_h4, cv_results_h4)

iteration: 0
heuristic 4 started
heuristic 4 started
heuristic 4 started
heuristic 4 started
heuristic 4 started

In [ ]:
def get_gens_from_descr(descr, gens):
    pairs = [p.replace("compare_expr(", "").replace(")", "").split(", ")
             for p in descr.split("\t")]
    gp = []
    for p1, p2 in pairs:
        gp.append((gens[int(p1)], gens[int(p2)]))
    return gp   

def pairs_to_str(pairs):
    pairs = ["{} {}".format(p1, p2) for p1, p2 in pairs]
    return "\t".join(pairs)
    

In [ ]:
with open("pairs.tsv", "w") as f:
    for descr in descriptions_t2[0]:
        f.write(pairs_to_str(get_gens_from_descr(descr, gens)))
        f.write("\n")

In [ ]:
np.random.seed(3267)
a_tr, b_tr, cv_results_tr, descriptions_tr  = do_simmetry_class_cv(xpressions,
                                                      responses, 
                                                      clf,
                                                      cv=cv, 
                                                      iters=10,
                                                      feature_maker=heuristics_4_features_rnd)
process_results(a_tr, b_tr, cv_results_tr)

In [ ]:
descr_set = set(descriptions_t[0][0].split("\t"))

for description in descriptions_t2:# + descriptions_t:
    for d in description:        
        descr = set(d.split("\t"))
        print(len(descr))
        descr_set = descr_set.intersection(descr)
    
    
print("\n", len(descr_set))
# process_results(a_t2 + a_t, b_t2 + b_t, cv_results_t2 + cv_results_t)

Классификация по пересечению генов из эвристик 1, 2:

In [ ]:
inter_h1_h2_gens = ['PDE6C', 'CEP57', 'PDE9A', 'CALU', 'TTK', 'AATF', 'MIR1244-1',
       'MIR1244-2', 'MIR1244-3', 'TMEM109', 'QKI', 'PDIA6', 'PDK1',
       'SNAPC3', 'BACE2', 'NAA50', 'TTYH1', 'EPHB3', 'RPIA', 'SF3B3',
       'GORASP2', 'NUP153', 'NUP155', 'LRRC59', 'PDSS1', 'HLA-G', 'BANK1',
       'ANP32E', 'FOCAD', 'XPNPEP1', 'FOLH1B', 'TUBB4B', 'RAB23', 'EPRS',
       'SUMO3', 'LSM2', 'IQCG', 'NUSAP1', 'NASP', 'CSNK2B', 'ABI1', 'PERP',
       'GPM6B', 'PES1', 'DNAJA1', 'SYK', 'CASP8AP2', 'HN1', 'CHI3L2',
       'ACACA', 'RPL39L', 'TXNDC5', 'BCL11A', 'ISG20', 'SYNRG', 'TYMS',
       'RPP21', 'HNRNPH3', 'RAD1', 'ESPL1', 'CHRM3', 'ITGA6', 'RPS21',
       'ACP1', 'GPR89A', 'GPR89B', 'LCN2', 'CTPS1', 'ITGB4', 'SNRPC',
       'SNRPD1', 'NDC80', 'MAD2L1', 'GPSM2', 'PHACTR2', 'UBE2J1',
       'RALGAPB', 'TAF1D', 'NDRG2', 'TAF5', 'DONSON', 'ITPR3', 'RPS7',
       'TNFAIP8L2-SCNM1', 'RANBP6', 'BLOC1S5-TXNDC5', 'TAP1', 'RAP2A',
       'RRAGD', 'ACTR3', 'ACTR3B', 'HPS5', 'RRM2', 'FZD7', 'SKP2', 'DPY30',
       'CCHCR1', 'NDUFB6', 'CKS1B', 'CKS2', 'MAGOHB', 'DROSHA', 'RASGRP3',
       'KARS', 'DSC2', 'SOX10', 'UGP2', 'ZIC1', 'CYB5R2', 'CCNC', 'TNS2',
       'CCND3', 'NELFB', 'HSPA14', 'TBCC', 'FAIM2', 'BRIX1', 'BRPF1',
       'FAM107A', 'TOMM70', 'NES', 'DUSP14', 'DUSP2', 'CLIC4', 'RBM28',
       'GTF2E2', 'MAP4K4', 'S100A1', 'DUSP6', 'ZNF131', 'CCT4', 'NT5DC2',
       'CCT6A', 'MAPK1', 'BTG3', 'NFIB', 'UQCRH', 'UQCRHL', 'PRPF3',
       'GTPBP8', 'SAC3D1', 'NFYA', 'PRPF40A', 'PRPF4B', 'TCF7L2', 'CMC2',
       'KCNN4', 'CMPK1', 'PRPS2', 'USP18', 'PABPC3', 'MRPL9', 'LMO4',
       'SAP30', 'GATA6', 'GYG1', 'MRPS17', 'PADI2', 'MRPS18A', 'MRPS2',
       'USP39', 'ADSL', 'PRRC2A', 'SPR', 'GBP1', 'ART3', 'TEAD4',
       'PAK1IP1', 'PLCB4', 'MSH2', 'HACD1', 'IFI16', 'LOC100506248',
       'HADHA', 'MSH6', 'IFI44L', 'RFC2', 'LOC100506639', 'RFC4', 'COCH',
       'NMI', 'TRDMT1', 'FAM98A', 'C1orf112', 'SREK1', 'FANCG', 'PSIP1',
       'RFWD3', 'FANCL', 'MCM3', 'VCP', 'MCM6', 'TFDP2', 'CDC45', 'PSMB4',
       'TFF2', 'EFNA4', 'PLK4', 'VEGFA', 'CDCA8', 'SLC43A3', 'SCNM1',
       'EFS', 'SRSF3', 'VGLL1', 'GGH', 'SCPEP1', 'VGLL4', 'KIF11', 'MEA1',
       'TRIM38', 'TRIM39', 'TRIM39-RPP21', 'KIF14', 'KIF15', 'FBXO11',
       'DDX18', 'MTMR2', 'PSME4', 'PSMG1', 'SSR1', 'MED17', 'KIF20A',
       'HDAC9', 'MTO1', 'HDGF', 'PMPCA', 'SEC13', 'TRMT11', 'VPS72',
       'MELK', 'MEMO1', 'CDKN2A', 'C6orf62', 'ATP5G3', 'FDPS', 'DEK',
       'CDR2L', 'KLF11', 'IL12RB2', 'STAG1', 'FEN1', 'SEL1L3', 'DENND4B',
       'EIF5B', 'PTMA', 'KLF6', 'LOC389906', 'ELAVL1', 'KLHDC3', 'HIC2',
       'MEX3C', 'MYCN', 'ELF5', 'ZNF74', 'MFGE8', 'CEBPG', 'GMDS', 'CECR5',
       'STEAP1B', 'GMIP', 'LOC728026', 'RNF138', 'TSPAN6', 'TSPO', 'WDR77',
       'SERBP1', 'NRTN', 'SMC5', 'MYO10', 'STMN1', 'CENPM', 'DHTKD1',
       'ILF2', 'STOML2', 'LPIN1', 'PUS1', 'MICAL3', 'TTC13', 'PWP2',
       'CRLF1']

In [ ]:
ids = np.arange(gens.shape[0])
interscetion_h1_h2 = sorted([ids[gen==gens][0] for gen in inter_h1_h2_gens if (gen == gens).sum() == 1])
# print(interscetion_h1_h2)

np.random.seed(3267)
a, b, cv_results = do_simmetry_class_cv(xpressions_scaled[:, interscetion_h1_h2], inv_responses, clf, cv=cv)
process_results(a, b, cv_results)[[2,3,0,1,4,5]]

In [ ]:
len(inter_h1_h2_gens)

In [ ]:
np.random.seed(3267)
knn = KNeighborsClassifier(n_neighbors=1)
a,b, cv_results  = do_simmetry_class_cv(xpressions_scaled[:, interscetion_h1_h2], inv_responses, knn, cv=cv)
process_results(a, b, cv_results)[[2,3,0,1,4,5]]

Случайно отобранные гены

In [ ]:
np.random.seed(3267)
rnd_h1_h2 = np.random.choice(ids, size=len(interscetion_h1_h2), replace=False)
a, b, cv_results = do_simmetry_class_cv(xpressions_scaled[:, rnd_h1_h2], inv_responses, clf, cv=cv)
process_results(a, b, cv_results)[[2,3,0,1,4,5]]

In [ ]:
np.random.seed(3267)
knn = KNeighborsClassifier(n_neighbors=1)
a,b, cv_results  = do_simmetry_class_cv(xpressions_scaled[:, rnd_h1_h2], inv_responses, knn, cv=cv)
process_results(a, b, cv_results)[[2,3,0,1,4,5]]

Классификация по 70 парам из эвристики 4

In [ ]:
good_pairs_str = [('APBB3', 'CRLF1'), ('ARHGAP1', 'CRLF1'), ('ATP6V0A1', 'IL12RB2'), ('ATP6V0C', 'DUSP14'), ('C4A', 'NKTR'),
              ('C4B', 'NKTR'), ('C4B_2', 'NKTR'), ('CCND1', 'BCOR'), ('CCND1', 'MON1B'), ('CCND1', 'MTRF1'), ('CCND1', 'ZNF165'),
              ('CDK16', 'COCH'), ('CERS6', 'CRIPT'), ('CERS6', 'JMJD1C'), ('CERS6', 'MTO1'), ('CERS6', 'TLE1'),
              ('CERS6', 'TOPORS-AS1'), ('COL1A1', 'FOLH1B'), ('DHCR24', 'COCH'), ('DHCR24', 'FOCAD'), ('DNASE2', 'GPM6B'),
              ('EIF4B', 'ITGA6'), ('ERC1', 'CRLF1'), ('FHOD1', 'CRLF1'), ('FILIP1L', 'CRLF1'), ('FKSG49', 'MFGE8'),
              ('FOLR1', 'COCH'), ('GOLGA1', 'CRLF1'), ('GON4L', 'CRLF1'), ('GP1BA', 'CRLF1'), ('HOXB5', 'CRLF1'),
              ('HOXC4', 'FAIM2'), ('IGFBP4', 'DEPDC5'), ('IGFBP4', 'PIK3CG'), ('LYST', 'KCNAB1'), ('MIR6831', 'CRLF1'),
              ('MOSPD2', 'CRLF1'), ('MXRA8', 'GATA6'), ('MYL4', 'CRLF1'), ('MZT2B', 'ITGA6'), ('NAT1', 'PAQR3'),
              ('NDUFB5', 'MELK'), ('NECAB3', 'FBXO11'), ('NXPH3', 'CRLF1'), ('PEA15', 'NFIB'), ('PLXNA3', 'CRLF1'),
              ('PNPO', 'RWDD3'), ('POLDIP2', 'COCH'), ('PPIE', 'IL12RB2'), ('R3HDM4', 'ITGA6'), ('RALA', 'GPM6B'),
              ('RASA1', 'YES1'), ('RUFY1', 'ITGA6'), ('SEMA3C', 'BCOR'), ('SEMA3C', 'YIPF4'), ('SNORA67', 'ITGA6'),
              ('SPR', 'DNPH1'), ('STAMBP', 'MSH2'), ('SULT1A2', 'MRPS2'), ('THEMIS2', 'CRLF1'), ('TM6SF1', 'ELAVL2'),
              ('TMEM62', 'ETAA1'), ('TNK1', 'FBXO11'), ('TPBG', 'COCH'), ('TPBG', 'WSB1'), ('TRANK1', 'CRLF1'),
              ('TRPM2', 'CRLF1'), ('VPS35', 'MELK'), ('ZDHHC7', 'ITGA6'), ('ZNF510', 'CRLF1')]
gens_list = gens.tolist()
good_pairs = [(gens_list.index(gen1), gens_list.index(gen2)) for gen1, gen2 in good_pairs_str
              if (gen1 in gens_list) and (gen2 in gens_list)]

order_features1, pair_names = xpression_to_order_feature(xpressions, good_pairs, gens)
print(order_features1.shape)

np.random.seed(3267)
a, b, cv_results = do_simmetry_class_cv(order_features1, inv_responses, clf, cv=cv)
process_results(a, b, cv_results)[[2,3,0,1,4,5]]

In [ ]:
np.random.seed(3267)
knn = KNeighborsClassifier(n_neighbors=1)
a,b, cv_results  = do_simmetry_class_cv(order_features1, inv_responses, knn, cv=cv)
process_results(a, b, cv_results)[[2,3,0,1,4,5]]

In [ ]:
def get_splits(responses, k=10):
    nonresp = np.arange(responses.shape[0])[responses==0]
    resp = np.arange(responses.shape[0])[responses==1]
    np.random.shuffle(nonresp)
    np.random.shuffle(resp)
    folds = []
    for i in range(9):
        folds.append(np.hstack((resp[i*22:i*22+22],nonresp[i*5:i*5+5])))
    folds.append(np.hstack((resp[9*22:],nonresp[9*5:])))
    cases = []
    for i in range(10):
        test = folds[i]
        train = np.hstack((fold for j, fold in enumerate(folds) if i != j))
        cases.append( (train, test) )                      
    return folds, cases
    

In [ ]:
folds, cases = get_splits(responses)

In [ ]:
intr_svc_smtr_a, intr_svc_smtr_b = do_simmetry_class_cv(order_features2,
                                                      responses, svc, iters=10)
order_features.shape, order_features2.shape, order_features3.shape

In [ ]:
print(intr_svc_smtr_a.mean(axis=0), intr_svc_smtr_b.mean(axis=0))
print()
# print(best31_svc_smtr_a.mean(axis=0), best31_svc_smtr_b.mean(axis=0))
# prec_0, recall_0, prec_1, recall_1

In [ ]:
print(intr_svc_smtr_a.mean(axis=0), intr_svc_smtr_b.mean(axis=0))
print()
# print(best31_svc_smtr_a.mean(axis=0), best31_svc_smtr_b.mean(axis=0))
# prec_0, recall_0, prec_1, recall_1

In [ ]:
import gensim